In [1]:
#importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import gmsh
import time
import pylab as pl
import gmsh
from IPython import display
from matplotlib.path import Path
from scipy.sparse import coo_array
from collections import Counter
%matplotlib inline
import scipy

In [2]:
def read_mesh(filepath):
    '''
    Takes in an msh file and should return nodal coordinates and element connectivity in each physical group along with the boundary nodes 
    '''
    gmsh.initialize()
    gmsh.open(filepath)
    print(f"Reading {filepath}")
    print(f"Number of nodes in the mesh: {int(gmsh.option.getNumber("Mesh.NbNodes"))}")
    print(f"Number of triangles in the mesh: {int(gmsh.option.getNumber("Mesh.NbTriangles"))}\n")

    #Get all nodes
    dim = -1
    tag = -1
    nodeTags, nodecoords, _ = gmsh.model.mesh.getNodes(dim,tag)
    nodecoords = nodecoords.reshape(-1,3) #tags start from 1

    #Get all triangles
    eleType = 2
    tag = -1
    elements_t,ele_con = gmsh.model.mesh.getElementsByType(eleType,-1)
    ele_con = ele_con.reshape(-1,3)  #tags start from 1


    gmsh.finalize()
    return [nodecoords,ele_con] 

In [27]:
def Q(point,centre,ro):
    x = point[0,0] - centre[0,0]
    y = point[0,1] - centre[0,1]
    Qo = 5 ## amplitude in W/mm^2 
    return Qo*np.exp(-(x**2+y**2)/ro**2)  ## W/m^3

In [36]:
class FEM:
    def __init__(self):
        self.data = {"ips":{1:[[1/3,1/3]],3:[[1/6,1/6],[1/6,2/3],[2/3,1/6]]},\
       "weights":{1:[1/2],3:[1/6,1/6,1/6]}}
                        
    def fit_ele(self,nodes,ele,centre):
        '''
        Return mass and stiffness matrices alongside the forcing vector
        '''
        gp = 3

        qo = 1e-3   # W/mm^2
        c = 648 #J/kg.K
        rho = 7.6e6 #kg/mm^3
        k = 0.025 #W/mm.K
        ro = 2 #mm
        vo = 2 #mm/s
        
        #getting in the required data and doing the initialization
        nop = nodes.shape[0]
        ips = np.array(self.data["ips"][gp])
        weights = np.array(self.data["weights"][gp])

        K = np.zeros((nop,nop))
        G = np.zeros((nop,nop))
        F = np.zeros((nop,1))

        for i,elei in enumerate(ele):
            econ = elei-1
            boundary = nodes[np.ix_(econ,[0,1])]
            dN = np.array([[-1,1,0],[-1,0,1]])
            Jac = np.matmul(dN,boundary)
            if np.linalg.det(Jac)<0:
                econ[0],econ[1] = econ[1],econ[0] #reordering for the direction to be counter clockwise
                boundary = nodes[np.ix_(econ,[0,1])] 
                Jac = np.matmul(dN,boundary)
            dN_dx = (np.linalg.inv(Jac)@dN)[0].reshape(1,-1) #1x3
            Jac_inv = np.linalg.inv(Jac)
            for j,ipj in enumerate(ips):
                N = np.array([[(1-ipj[0]-ipj[1]), ipj[0],ipj[1]]])
                a = (Jac_inv@dN).T@(Jac_inv@dN)*(np.linalg.det(Jac))*weights[j]
                b = N.T@dN_dx*(np.linalg.det(Jac))*weights[j]
                K[np.ix_(econ,econ)] += k*a
                G[np.ix_(econ,econ)] += rho*c*vo*b
                X  =np.matmul(N,boundary)
                f = N*Q(X,centre,ro)*np.linalg.det(Jac)*weights[j]
                F[np.ix_(econ,[0])] +=f.T
        print("K\n",K.round(2))
        print("G\n",G.round(2))
        print("F\n",F.round(2))
        return [K,G,F]


In [37]:
soln1 = FEM()

In [38]:
mesh_size_factor = 10

In [39]:
filename = f"rectangle_{mesh_size_factor}.msh"
nodecoords,ele_con = read_mesh(filename) #node tags start from 1
K,G,F = soln1.fit_ele(nodecoords,ele_con,np.array([[50,25]]))

Reading rectangle_10.msh
Number of nodes in the mesh: 79
Number of triangles in the mesh: 126

K
 [[ 0.02  0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.02  0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.02 ... -0.01  0.    0.  ]
 ...
 [ 0.    0.   -0.01 ...  0.09  0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.09  0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.09]]
G
 [[-1.6416e+10  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
   0.0000e+00]
 [ 0.0000e+00  1.6416e+10  0.0000e+00 ...  0.0000e+00  0.0000e+00
   0.0000e+00]
 [ 0.0000e+00  0.0000e+00  1.6416e+10 ... -1.6416e+10  0.0000e+00
   0.0000e+00]
 ...
 [ 0.0000e+00  0.0000e+00  1.6416e+10 ... -0.0000e+00  0.0000e+00
   0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00 ...  0.0000e+00 -0.0000e+00
   0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
  -0.0000e+00]]
F
 [[0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+0

In [41]:
## left hand side 20 degree dirichlet
T_l = 273+20
left_nodes = np.where(nodecoords[:,0] == 0)[0]
right_nodes = np.where(nodecoords[:,0] == np.max(nodecoords[:,0]))[0]
bottom_nodes = np.where(nodecoords[:,1] == 0)[0]
top_nodes = np.where(nodecoords[:,1] == np.max(nodecoords[:,1]))[0]